In [2]:
# 1. Data analysis

# 1. Importar las librerías necesarias
library(readr)
library(dplyr)
library(tidyr)

# Ruta del archivo
file_path <- "C:/Users/alessandro/Desktop/Tarea 1/wage2015_subsample_inference.csv"

# Cargar los datos
data <- read_csv(file_path)

# Ver las primeras filas y nombres de las columnas
head(data)
colnames(data)

# 2. Verificar valores faltantes por cada columna
missing_values <- colSums(is.na(data))
print(missing_values)

# Mensaje sobre valores faltantes
if (any(missing_values > 0)) {
  cat("Las siguientes columnas tienen valores faltantes:\n")
  print(missing_values[missing_values > 0])
} else {
  cat("No hay valores faltantes en el conjunto de datos.\n")
}

# 3. Separar variables numéricas y categóricas
numeric_vars <- select(data, wage, lwage, exp1, exp2, exp3, exp4)
categorical_vars <- select(data, sex, shs, hsg, scl, clg, ad, mw, so, we, ne, occ2, ind2)

# Calcular estadísticas descriptivas para variables numéricas
descriptive_stats_numeric <- numeric_vars %>%
  summarise(across(everything(), list(
    media = ~mean(., na.rm = TRUE), 
    desviacion = ~sd(., na.rm = TRUE), 
    mediana = ~median(., na.rm = TRUE), 
    p25 = ~quantile(., 0.25, na.rm = TRUE), 
    p75 = ~quantile(., 0.75, na.rm = TRUE)
  ))) %>%
  pivot_longer(everything(), names_to = c("variable", ".value"), names_sep = "_")

# Calcular frecuencias para variables categóricas
# Crear una lista para almacenar los resultados
categorical_freq_list <- list()

for (var in names(categorical_vars)) {
  freq_table <- categorical_vars %>%
    count(!!sym(var)) %>%
    rename(categoria = !!sym(var), frecuencia = n) %>%
    mutate(variable = var)  # Agregar el nombre de la variable
  categorical_freq_list[[var]] <- freq_table
}

# Combinar todas las tablas de frecuencias en un solo dataframe
descriptive_stats_categorical <- bind_rows(categorical_freq_list)

# Mostrar las estadísticas descriptivas
cat("Estadísticas Descriptivas para Variables Numéricas:\n")
print(descriptive_stats_numeric)

cat("\nFrecuencias para Variables Categóricas:\n")
print(descriptive_stats_categorical)

# 4. Filtrar el conjunto de datos para mujeres con un título universitario o superior
filtered_data <- data %>%
  filter(sex == 1 & (clg == 1 | ad == 1))  # sex == 1 indica mujeres; clg == 1 o ad == 1 para títulos universitarios

# Calcular el umbral del 25% más alto de salarios
wage_threshold <- quantile(data$wage, 0.75, na.rm = TRUE)  # 75% para obtener el umbral del 25% más alto

# Filtrar el conjunto de datos nuevamente para incluir solo a las mujeres que ganan por encima del umbral
rich_women <- filtered_data %>%
  filter(wage >= wage_threshold)

# Contar cuántas mujeres cumplen con las condiciones y mostrar el dataframe
num_rich_women <- nrow(rich_women)

# Mostrar el dataframe de mujeres con salarios en el 25% más alto
print(rich_women)

# Mostrar el número total de mujeres en esta categoría
print(paste("Número de mujeres con título universitario o superior y salario en el 25% más alto:", num_rich_women))

# 5. Filtrar el conjunto de datos para hombres con un título de secundaria o inferior
filtered_data_men <- data %>%
  filter(sex == 0 & (hsg == 1 | shs == 1))  # sex == 0 indica hombres; hsg == 1 o shs == 1 para títulos de secundaria

# Calcular el umbral del 25% más alto de salarios
wage_threshold_men <- quantile(data$wage, 0.75, na.rm = TRUE)  # 75% para obtener el umbral del 25% más alto

# Filtrar el conjunto de datos nuevamente para incluir solo a los hombres que ganan por encima del umbral
rich_men <- filtered_data_men %>%
  filter(wage >= wage_threshold_men)

# Contar cuántos hombres cumplen con las condiciones y mostrar el dataframe
num_rich_men <- nrow(rich_men)

# Mostrar el dataframe de hombres con salarios en el 25% más alto
print(rich_men)

# Mostrar el número total de hombres en esta categoría
print(paste("Número de hombres con título de secundaria o inferior y salario en el 25% más alto:", num_rich_men))

# 6. Crear un dataframe con solo log(wage)
log_wage_df <- data %>%
  select(lwage)  # Solo selecciona la columna lwage

# Crear un dataframe con todas las variables excepto las relacionadas con el salario
data_no_wage_df <- data %>%
  select(-wage, -lwage)  # Excluye las columnas wage y lwage

# Mostrar los dataframes creados
print("Dataframe con solo log(wage):")
print(log_wage_df)

print("Dataframe sin variables relacionadas con el salario:")
print(data_no_wage_df)

# 2. Data wrangling

# 7. Crear un array para la variable Y (log(wage))
Y_array <- as.array(data$lwage)

# Mostrar el array creado
print("Array de la variable Y (log(wage)):")
print(Y_array)

# 8.1. Crear el Array para el Modelo Básico

# Crear variables dummy para las variables categóricas utilizando model.matrix
dummy_vars <- model.matrix(~ sex + exp1 + hsg + scl + clg + ad + so + we + ne + occ2 + ind2, data = data)

# Excluir la primera dummy para evitar multicolinealidad
dummy_vars <- dummy_vars[, -1]  # Excluir la primera columna que corresponde al intercepto

# Convertir a array
basic_model_array <- as.matrix(dummy_vars)

# Mostrar el array del modelo básico
print("Array del modelo básico:")
print(basic_model_array)

# 8.2 Crear el Array para el Modelo Flexible

# Agregar variables polinómicas de exp
data$exp2 <- data$exp1^2
data$exp3 <- data$exp1^3
data$exp4 <- data$exp1^4

# Crear variables dummy para las variables categóricas nuevamente
dummy_vars_flexible <- model.matrix(~ exp1 + exp2 + exp3 + exp4 + hsg + scl + clg + ad + so + we + ne + occ2 + ind2, data = data)

# Excluir la primera dummy para evitar multicolinealidad
dummy_vars_flexible <- dummy_vars_flexible[, -1]

# Crear interacciones entre las variables de experiencia y las demás variables (excepto sex)
for (var in c("hsg", "scl", "clg", "ad", "so", "we", "ne", "occ2", "ind2")) {
  for (exp_var in c("exp1", "exp2", "exp3", "exp4")) {
    dummy_vars_flexible <- cbind(dummy_vars_flexible, data[[exp_var]] * dummy_vars_flexible[, var, drop = FALSE])
    colnames(dummy_vars_flexible)[ncol(dummy_vars_flexible)] <- paste(exp_var, var, "interaction", sep = "_")
  }
}

# Convertir a array
flexible_model_array <- as.matrix(dummy_vars_flexible)

# Mostrar el array del modelo flexible
print("Array del modelo flexible:")
print(flexible_model_array)

# 8.3 Crear el Array para el Modelo Extra-Flexible

# Seleccionar las variables que queremos para el modelo extra-flexible
extra_flexible_vars <- data %>%
  select(exp1, exp2, exp3, exp4, hsg, scl, clg, ad, so, we, ne, starts_with("occ2_"), starts_with("ind2_"))

# Crear todas las interacciones de dos vías (excepto sex)
extra_flexible_interactions <- combn(names(extra_flexible_vars), 2, simplify = FALSE)

# Generar las interacciones
for (pair in extra_flexible_interactions) {
  extra_flexible_vars <- cbind(extra_flexible_vars, extra_flexible_vars[[pair[1]]] * extra_flexible_vars[[pair[2]]])
  colnames(extra_flexible_vars)[ncol(extra_flexible_vars)] <- paste(pair[1], pair[2], "interaction", sep = "_")
}

# Eliminar duplicados
extra_flexible_vars <- unique(extra_flexible_vars)

# Convertir a array
extra_flexible_model_array <- as.matrix(extra_flexible_vars)

# Mostrar el array del modelo extra-flexible
print("Array del modelo extra-flexible:")
print(extra_flexible_model_array)

# 9. NORMALIZAR 
# 1. Normalizar Y (lwage)
data$lwage_tilde <- (data$lwage - mean(data$lwage, na.rm = TRUE)) / sd(data$lwage, na.rm = TRUE)

# 2. Normalizar las variables numéricas
numeric_vars <- c("wage", "exp1", "exp2", "exp3", "exp4", "lwage_tilde")  # Incluye lwage_tilde
for (var in numeric_vars) {
  data[[paste0(var, "_tilde")]] <- (data[[var]] - mean(data[[var]], na.rm = TRUE)) / sd(data[[var]], na.rm = TRUE)
}

# 3. Normalizar las variables categóricas (si es necesario)
categorical_vars <- c("sex", "hsg", "scl", "clg", "ad", "so", "we", "ne")
for (var in categorical_vars) {
  data[[paste0(var, "_tilde")]] <- (as.numeric(data[[var]]) - mean(as.numeric(data[[var]]), na.rm = TRUE)) / sd(as.numeric(data[[var]]), na.rm = TRUE)
}

# Mostrar las primeras filas del dataframe con las variables normalizadas
head(data)


# 3. Linear Regressions

# 10. División de los DataFrames para los Modelos

# 1. Definir la semilla para la reproducibilidad
set.seed(123)

# 2. Crear índices para el entrenamiento (80%) y prueba (20%)
train_indices <- sample(1:nrow(data), size = 0.8 * nrow(data))

# 3. Dividir el dataframe en entrenamiento y prueba para el modelo básico
basic_model_data <- data[, c("lwage_tilde", "sex_tilde", "exp1_tilde", "hsg_tilde", "scl_tilde", "clg_tilde", "ad_tilde", "so", "we", "ne", "occ2", "ind2")]

basic_train <- basic_model_data[train_indices, ]
basic_test <- basic_model_data[-train_indices, ]

# Imprimir tamaños de las muestras para el modelo básico
cat("Tamaño de la muestra de entrenamiento (modelo básico):", nrow(basic_train), "\n")
cat("Tamaño de la muestra de prueba (modelo básico):", nrow(basic_test), "\n")

# 4. Dividir el dataframe en entrenamiento y prueba para el modelo flexible
flexible_model_data <- data[, c("lwage_tilde", "sex_tilde", "exp1_tilde", "exp2_tilde", "exp3_tilde", "exp4_tilde", "hsg_tilde", "scl_tilde", "clg_tilde", "ad_tilde", "so", "we", "ne", "occ2", "ind2")]

flexible_train <- flexible_model_data[train_indices, ]
flexible_test <- flexible_model_data[-train_indices, ]

# Imprimir tamaños de las muestras para el modelo flexible
cat("Tamaño de la muestra de entrenamiento (modelo flexible):", nrow(flexible_train), "\n")
cat("Tamaño de la muestra de prueba (modelo flexible):", nrow(flexible_test), "\n")

# 5. Dividir el dataframe en entrenamiento y prueba para el modelo extra-flexible
extra_flexible_model_data <- data[, c("lwage_tilde", "sex_tilde", "exp1_tilde", "exp2_tilde", "exp3_tilde", "exp4_tilde", "hsg_tilde", "scl_tilde", "clg_tilde", "ad_tilde", "so", "we", "ne", "occ2", "ind2")]

extra_flexible_train <- extra_flexible_model_data[train_indices, ]
extra_flexible_test <- extra_flexible_model_data[-train_indices, ]

# Imprimir tamaños de las muestras para el modelo extra-flexible
cat("Tamaño de la muestra de entrenamiento (modelo extra-flexible):", nrow(extra_flexible_train), "\n")
cat("Tamaño de la muestra de prueba (modelo extra-flexible):", nrow(extra_flexible_test), "\n")

# 11. Estimar los modelos

# Modelo básico
basic_model <- lm(lwage_tilde ~ sex_tilde + exp1_tilde + hsg_tilde + scl_tilde + 
                    clg_tilde + ad_tilde + so + we + ne + occ2 + ind2, 
                  data = basic_train)

# Modelo flexible
flexible_model <- lm(lwage_tilde ~ sex_tilde + exp1_tilde + exp2_tilde + exp3_tilde + 
                       exp4_tilde + hsg_tilde + scl_tilde + clg_tilde + ad_tilde + 
                       so + we + ne + occ2 + ind2, 
                     data = flexible_train)

# Modelo extra-flexible
extra_flexible_model <- lm(lwage_tilde ~ sex_tilde + exp1_tilde + exp2_tilde + exp3_tilde + 
                             exp4_tilde + hsg_tilde + scl_tilde + clg_tilde + ad_tilde + 
                             so + we + ne + occ2 + ind2, 
                           data = extra_flexible_train)

# Resumen de los modelos
summary(basic_model)
summary(flexible_model)
summary(extra_flexible_model)

# 12. the MSE, the R^2 y the adjusted R^2

calculate_metrics <- function(model, data, sample_name) {
  predictions <- predict(model, newdata = data)
  actuals <- data$lwage_tilde
  
  mse <- mean((predictions - actuals)^2)
  r_squared <- 1 - (sum((predictions - actuals)^2) / sum((actuals - mean(actuals))^2))
  adjusted_r_squared <- 1 - (1 - r_squared) * (nrow(data) - 1) / (nrow(data) - length(coef(model) - 1))
  
  return(c(MSE = mse, R_squared = r_squared, Adjusted_R_squared = adjusted_r_squared, Sample = sample_name))
}
# Modelo básico
basic_metrics_train <- calculate_metrics(basic_model, basic_train, "Basic - Train")
basic_metrics_test <- calculate_metrics(basic_model, basic_test, "Basic - Test")

# Modelo flexible
flexible_metrics_train <- calculate_metrics(flexible_model, flexible_train, "Flexible - Train")
flexible_metrics_test <- calculate_metrics(flexible_model, flexible_test, "Flexible - Test")

# Modelo extra-flexible
extra_flexible_metrics_train <- calculate_metrics(extra_flexible_model, extra_flexible_train, "Extra-flexible - Train")
extra_flexible_metrics_test <- calculate_metrics(extra_flexible_model, extra_flexible_test, "Extra-flexible - Test")

# Combinar los resultados
results <- rbind(basic_metrics_train, basic_metrics_test,
                 flexible_metrics_train, flexible_metrics_test,
                 extra_flexible_metrics_train, extra_flexible_metrics_test)

# Convertir a un data frame para mejor visualización
results_df <- as.data.frame(results)

# Mostrar resultados
print(results_df)



ERROR: Error in library(readr): there is no package called 'readr'
